# Pokémon TCG Scarlet & Violet 151 (English) Pack Analysis

## Introduction

The goal of this analysis is to determine whether it is worth opening a pack of Pokémon cards from the Scarlet & Violet 151 set. This includes an exploration of the expected value (EV) of opening a single pack based on the potential card pulls and their current market value.

---

## Understanding the Set

The **Scarlet & Violet 151** set is part of the Pokémon Trading Card Game series, featuring cards from the first 151 Pokémon of the franchise. The set includes a variety of card types:
- **Common**
- **Uncommon**
- **Rare**
- **Holo Rare**
- **Secret Rare**
- **Special Cards (e.g., Full Art, Alt Art)**

---

## Card Rarity Distribution

Understanding the rarity distribution within the pack helps estimate the probabilities of pulling specific types of cards:
- **Commons**: 5-6 cards per pack
- **Uncommons**: 2-3 cards per pack
- **Energy***: 1 card per pack
    - These **Energy** cards have 1/4 chance to a foil
- **Rare**: 1 card per pack
    - These **Rare** cards has as chance to be a **Holo Rare**, **Secret Rare**, **Special Cards (e.g., Full Art, Alt Art)**
- However, there is also a 1/8 chance that a pack contains a **Double Rare** where 2 **Rare** cards exist in a pack (with 1 lesser **Common** or **Uncommon** card)
---

## Expected Value Calculation

### Step 1: Market Value of Cards

Card values will be web scraped in real time from pricecharting.com to provide the latest card values based on market prices (e.g. Ebay, Amazon) without including any shipping costs

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver to simulate browser
driver = webdriver.Chrome()
driver.get("https://www.pricecharting.com/console/pokemon-japanese-scarlet-&-violet-151")

# Scroll to the bottom of the page multiple times to load all items
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the new content
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Detect whether page has ended
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Break if no new content is loaded
    last_height = new_height

# Once the full content is loaded, parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Scrape the data
products = soup.findAll("tr", {"id": lambda x: x and x.startswith("product-")})

In [2]:
# Extract and store data
data = []
for product in products:
    title_tag = product.find("td", class_="title")
    title = title_tag.text.strip() if title_tag else "No title found"
    
    if "#" in title:
        splitted = title.split("#")
        title = splitted[0].strip()
        card_number = int(splitted[-1].strip())
        
    else:
        card_number = 0
        
    if "[" in title:
        splitted = title.split("[")
        variant = splitted[-1][:-1].strip()
        title = splitted[0].strip()
    else:
        variant = "Normal"
    
    price_tag = product.find("span", class_="js-price")
    price = float(price_tag.text.strip()[1:]) if price_tag else 0
    
    data.append({"Title": title, "Card Number": card_number, "Price": price, "Variant": variant})

# Convert to DataFrame
SV = pd.DataFrame(data)
SV


,Title,Card Number,Price,Variant
0,Charizard EX,201,140.19,Normal
1,Pikachu,25,167.50,Master Ball
2,Mew EX,205,78.10,Normal
3,Pikachu,173,7.00,Normal
4,Gengar,94,199.00,Master Ball
...,...,...,...,...
345,Nidorina,30,1.87,Reverse
346,Cycling Road,165,2.33,Reverse
347,Nidorino,33,1.85,Reverse
348,Horsea,116,1.77,Reverse


### Step 2: Pull rates of Cards

